# TP3 - Views, Updates and Design

The objectives for this TP are:

1. Create and use SQL Views
2. Update database content
3. Design the database schema for a Social Network

___

For the first 2 parts we will use the **`wine.db`** database and the Tables created in TP2.

A reminder of the wine database schema:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [1]:
import sqlite3

In [2]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [3]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)

Database schema:
Clients2:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
Locations2:
  0: LIEU(TEXT)
  1: REGION(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
Producer1:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
Production1:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
RBB:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
Sales2:
  0: NV(NUM)
  1: NB(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
Wine1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
Wine2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


Again, we will use **`%%sql`** magic for our queries

In [4]:
%load_ext sql
%sql sqlite:///wine.db

u'Connected: @wine.db'

Recreate the Normalized Tables from **Master1** and **Master2** as you did in the TP2

In [5]:
%%sql
DROP TABLE IF EXISTS Wine1;

-- Create Wine1 table
CREATE TABLE Wine1 AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM Master1
WHERE NV IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [6]:
%%sql DROP TABLE IF EXISTS Producer1;

-- Create Producer1 table
CREATE TABLE Producer1 AS
SELECT DISTINCT NP, NOM, PRENOM, REGION
FROM Master1
WHERE NP IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [7]:
%%sql DROP TABLE IF EXISTS Production1;

-- Create Production1 table
CREATE TABLE Production1 AS
SELECT NP, NV, QTE
FROM Master1
WHERE NP IS NOT NULL and NV IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [8]:
%%sql DROP TABLE IF EXISTS Wine2;

-- Create Wine2 table
CREATE TABLE Wine2 AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM Master2
WHERE NV IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [9]:
%%sql DROP TABLE IF EXISTS Clients2;

-- Create Clients2 table
CREATE TABLE Clients2 AS
SELECT DISTINCT NB, NOM, PRENOM, TYPE
FROM Master2
WHERE NB IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [10]:
%%sql DROP TABLE IF EXISTS Sales2;

-- Create Sales2 table
CREATE TABLE Sales2 AS
SELECT NV, NB, DATES, LIEU, QTE
FROM Master2
WHERE NV IS NOT NULL and NB IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

In [11]:
%%sql DROP TABLE IF EXISTS Locations2;

-- Create Locations2 table
CREATE TABLE Locations2 AS
SELECT DISTINCT LIEU, REGION
FROM Master2
WHERE LIEU IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.


[]

___
# PART I: CREATE AND USE VIEWS

A view is a virtual table based on the result-set of an SQL statement. Views are stored in the database with an associated name.

Views are created following the syntax:

```mysql
CREATE VIEW view_name AS
SELECT column1, column2.....
FROM table_name
WHERE [condition];
```

An useful command:

```mysql
DROP VIEW IF EXISTS view_name;
```


__Warning:__ Use `DROP` with caution (only drop something if you are sure)

__Note:__ You will find some cells marked as "Test" that will help you check your work. Do NOT modify them. 

#### Exercise 1.1

Create a view "**bons_buveurs**" with the clients (buveurs) of type 'gros' or 'moyen'.

In [12]:
%%sql
DROP VIEW IF EXISTS bons_buveurs;

CREATE VIEW bons_buveurs AS
SELECT *
FROM Clients2
WHERE TYPE = "gros" OR TYPE = "moyen"

 * sqlite:///wine.db
Done.
Done.


[]

In [13]:
# Test
%sql SELECT * FROM bons_buveurs ORDER BY nb;

 * sqlite:///wine.db
Done.


[(2, u'Artaud', u'Antonin', u'moyen'),
 (3, u'Aron', u'Raymond', u'gros'),
 (4, u'Apollinaire', u'Guillaume', u'moyen'),
 (6, u'Arrabal', u'Fernando', u'gros'),
 (7, u'Anouilh', u'Jean', u'moyen'),
 (8, u'Aragon', u'Louis', u'gros'),
 (10, u'Andersen', u'Yann', u'gros'),
 (12, u'Bataille', u'Georges', u'moyen'),
 (13, u'Barthes', u'Roland', u'moyen'),
 (14, u'Bory', u'Jean Louis', u'gros'),
 (16, u'Balzac', u'Honore de', u'moyen'),
 (18, u'Celine', u'Louis Ferdinand', u'gros'),
 (19, u'Ciceron', None, u'gros'),
 (20, u'Chateaubriand', u'Francois-Rene de', u'moyen'),
 (22, u'Cocteau', u'Jean', u'gros'),
 (24, u'Courrier', u'Paul Louis', u'moyen'),
 (26, u'Diderot', u'Denis', u'moyen'),
 (27, u'Dumas', u'Alexandre', u'gros'),
 (28, u'Dostoievsky', u'Fiodor', u'gros'),
 (31, u'Dante', None, u'gros'),
 (32, u'Eluard', u'Paul', u'moyen'),
 (33, u'Flaubert', u'Gustave', u'moyen'),
 (35, u'Fromentin', u'Eugene', u'gros'),
 (39, u'Montesquieu', None, u'gros'),
 (42, u'Goethe', u'Johann Wolfgang von', u'moyen'),
 (43, u'Musset', u'Alfred de', u'gros'),
 (46, u'Hugo', u'Victor', u'moyen'),
 (48, u'Hardy', u'Thomas', u'moyen'),
 (50, u'Lautreamont', None, u'moyen'),
 (53, u'Lamartine', u'Alphonse de', u'gros'),
 (55, u'Michelet', u'Jules', u'gros'),
 (57, u'Marx', u'Karl', u'gros'),
 (58, u'Marivaux', u'Pierre Carlet de Chamblain de', u'gros'),
 (60, u'Ovide', None, u'gros'),
 (62, u'Proust', u'Marcel', u'moyen'),
 (64, u'Platon', None, u'gros'),
 (66, u'Perec', u'Georges', u'gros'),
 (68, u'Prevert', u'Jacques', u'gros'),
 (70, u'Paulhan', u'Jean', u'gros'),
 (72, u'Rabelais', u'Francois', u'gros'),
 (73, u'Poe', u'Allan Edgard', u'moyen'),
 (75, u'Ronsard', u'Pierre', u'moyen'),
 (76, u'Racine', u'Jean', u'gros'),
 (78, u'Sandier', u'Jules', u'moyen'),
 (79, u'Sade', u'Donatien Alphonse Fran\xbf\xbfs de', u'gros'),
 (81, u'Sartre', u'Jean-Paul', u'gros'),
 (83, u'Sophocle', None, u'gros'),
 (84, u'Sue', u'Eugene', u'moyen'),
 (86, u'Saint Exupery', u'Antoine de', u'gros'),
 (87, u'Segur', u'Comtesse de', u'gros'),
 (91, u'Verlaine', u'Paul', u'gros'),
 (93, u'Virgile', None, u'gros'),
 (94, u'Zola', u'Emile', u'moyen'),
 (95, u'Zorn', u'Fritz', u'moyen'),
 (98, u'Maturin', u'Robert', u'gros'),
 (99, u'Cesar', u'Jules', u'gros')]

#### Exercise 1.2

Create the view "**buveurs_asec**" with clients (buveurs) who have not bought any wine.

In [14]:
%%sql
DROP VIEW IF EXISTS buveurs_asec;

CREATE VIEW buveurs_asec AS
SELECT DISTINCT NB, NOM, PRENOM, TYPE
FROM Clients2
WHERE NB NOT IN (SELECT DISTINCT NB
                 FROM Sales2)

 * sqlite:///wine.db
Done.
Done.


[]

In [15]:
# Test
%sql SELECT * FROM buveurs_asec ORDER BY nb;

 * sqlite:///wine.db
Done.


[(11, u'Breton', u'Andre', u'petit'),
 (13, u'Barthes', u'Roland', u'moyen'),
 (16, u'Balzac', u'Honore de', u'moyen'),
 (18, u'Celine', u'Louis Ferdinand', u'gros'),
 (20, u'Chateaubriand', u'Francois-Rene de', u'moyen'),
 (21, u'Corbiere', u'Tristan', u'petit'),
 (23, u'Corneille', u'Pierre', u'petit'),
 (25, u'Char', u'Rene', u'petit'),
 (27, u'Dumas', u'Alexandre', u'gros'),
 (29, u'Fournier', u'Alain', u'petit'),
 (32, u'Eluard', u'Paul', u'moyen'),
 (34, u'Mauriac', u'Francois', u'petit'),
 (35, u'Fromentin', u'Eugene', u'gros'),
 (36, u'Maurois', u'Andre', u'petit'),
 (37, u'Freud', u'Sigmund', u'petit'),
 (39, u'Montesquieu', None, u'gros'),
 (42, u'Goethe', u'Johann Wolfgang von', u'moyen'),
 (43, u'Musset', u'Alfred de', u'gros'),
 (46, u'Hugo', u'Victor', u'moyen'),
 (47, u'Gary', u'Romain', u'petit'),
 (51, u'Lafargue', u'Paul', u'petit'),
 (53, u'Lamartine', u'Alphonse de', u'gros'),
 (54, u'Aurele', u'Marc', u'petit'),
 (55, u'Michelet', u'Jules', u'gros'),
 (56, u'Nerval', u'Gerard de', u'petit'),
 (57, u'Marx', u'Karl', u'gros'),
 (59, u'Nietzsche', u'Friedrich', u'petit'),
 (63, u'Maupassant', u'Guy de', u'petit'),
 (64, u'Platon', None, u'gros'),
 (65, u'Pascal', u'Blaise', u'petit'),
 (66, u'Perec', u'Georges', u'gros'),
 (67, u'Nin', u'Anais', u'petit'),
 (68, u'Prevert', u'Jacques', u'gros'),
 (69, u'Queneau', u'Raymond', u'petit'),
 (70, u'Paulhan', u'Jean', u'gros'),
 (71, u'Radiguet', u'Raymond', u'petit'),
 (72, u'Rabelais', u'Francois', u'gros'),
 (73, u'Poe', u'Allan Edgard', u'moyen'),
 (74, u'Rimbaud', u'Arthur', u'petit'),
 (76, u'Racine', u'Jean', u'gros'),
 (77, u'Rousseau', u'Jean-Jacques', u'petit'),
 (78, u'Sandier', u'Jules', u'moyen'),
 (79, u'Sade', u'Donatien Alphonse Fran\xbf\xbfs de', u'gros'),
 (81, u'Sartre', u'Jean-Paul', u'gros'),
 (83, u'Sophocle', None, u'gros'),
 (84, u'Sue', u'Eugene', u'moyen'),
 (85, u'Segalen', u'Victor', u'petit'),
 (86, u'Saint Exupery', u'Antoine de', u'gros'),
 (87, u'Segur', u'Comtesse de', u'gros'),
 (88, u'Shakespeare', u'William', u'petit'),
 (89, u'Stendhal', None, u'petit'),
 (90, u'Vian', u'Boris', u'petit'),
 (91, u'Verlaine', u'Paul', u'gros'),
 (93, u'Virgile', None, u'gros'),
 (95, u'Zorn', u'Fritz', u'moyen'),
 (98, u'Maturin', u'Robert', u'gros'),
 (99, u'Cesar', u'Jules', u'gros')]

#### Exercise 1.3

Create the view "**buveurs_achats**" complementary to the previous one.

In [16]:
%%sql
DROP VIEW IF EXISTS buveurs_achats;

CREATE VIEW buveurs_achats AS
SELECT *
FROM Clients2
WHERE NB NOT IN (SELECT NB
                 FROM buveurs_asec)

 * sqlite:///wine.db
Done.
Done.


[]

In [17]:
# Test
%sql SELECT * FROM buveurs_achats ORDER BY nb;

 * sqlite:///wine.db
Done.


[(1, u'Aristote', None, u'petit'),
 (2, u'Artaud', u'Antonin', u'moyen'),
 (3, u'Aron', u'Raymond', u'gros'),
 (4, u'Apollinaire', u'Guillaume', u'moyen'),
 (5, u'Audiberti', u'Jacques', u'petit'),
 (6, u'Arrabal', u'Fernando', u'gros'),
 (7, u'Anouilh', u'Jean', u'moyen'),
 (8, u'Aragon', u'Louis', u'gros'),
 (9, u'Ajar', u'Emile', u'petit'),
 (10, u'Andersen', u'Yann', u'gros'),
 (12, u'Bataille', u'Georges', u'moyen'),
 (14, u'Bory', u'Jean Louis', u'gros'),
 (15, u'Baudelaire', u'Charles', u'petit'),
 (17, u'Camus', u'Albert', u'petit'),
 (19, u'Ciceron', None, u'gros'),
 (22, u'Cocteau', u'Jean', u'gros'),
 (24, u'Courrier', u'Paul Louis', u'moyen'),
 (26, u'Diderot', u'Denis', u'moyen'),
 (28, u'Dostoievsky', u'Fiodor', u'gros'),
 (30, u'La Fontaine', u'Jean de', u'petit'),
 (31, u'Dante', None, u'gros'),
 (33, u'Flaubert', u'Gustave', u'moyen'),
 (38, u'Montaigne', u'Michel de', u'petit'),
 (40, u'Moliere', None, u'petit'),
 (41, u'Gauthier', u'Theophile', u'petit'),
 (44, u'Gide', u'Andre', u'petit'),
 (45, u'Giono', u'Jean', u'petit'),
 (48, u'Hardy', u'Thomas', u'moyen'),
 (49, u'Kafka', u'Franz', u'petit'),
 (50, u'Lautreamont', None, u'moyen'),
 (52, u'Carroll', u'Lewis', u'petit'),
 (58, u'Marivaux', u'Pierre Carlet de Chamblain de', u'gros'),
 (60, u'Ovide', None, u'gros'),
 (61, u'Mallarme', u'Stephane', u'petit'),
 (62, u'Proust', u'Marcel', u'moyen'),
 (75, u'Ronsard', u'Pierre', u'moyen'),
 (80, u'Sand', u'Georges', u'petit'),
 (82, u'Roussel', u'Rene', u'petit'),
 (92, u'Verne', u'Jules', u'petit'),
 (94, u'Zola', u'Emile', u'moyen'),
 (96, u'Wilde', u'Oscar', u'petit'),
 (97, u'Vigny', u'Alfred de', u'petit'),
 (100, u'Woolf', u'Virginia', u'petit')]

#### Exercise 1.4

Create the view "**q83pl**" (LIEU, CRU, QTE_BUE) that provides by LIEU and CRU the total quantities bought in 1983 by all the clients (buveurs).

In [18]:
%%sql
DROP VIEW IF EXISTS q83pl;

CREATE VIEW q83pl AS
SELECT LIEU,CRU, SUM(QTE) as QTE_BUE
FROM Sales2
NATURAL JOIN Wine2
WHERE DATES LIKE "1983%"
GROUP BY LIEU, CRU

 * sqlite:///wine.db
Done.
Done.


[]

In [19]:
# Test
%sql SELECT * FROM q83pl;

 * sqlite:///wine.db
Done.


[(u'CAEN', u'Seyssel', 3),
 (u'LILLE', u'Pommard', 5),
 (u'LYON', u'Beaujolais Villages', 10),
 (u'LYON', u'Julienas', 2),
 (u'PARIS', u'Beaujolais Primeur', 4),
 (u'PARIS', u'Coteaux du Tricastin', 1),
 (u'PARIS', u'Pouilly Vinzelles', 3),
 (u'RENNES', u'Mercurey', 1),
 (u'ROCQUENCOURT', u'Beaujolais Villages', 260),
 (u'ROCQUENCOURT', u'Saint Amour', 80)]

#### Exercise 1.5

Can we define the same view with ascending order over the attribute "QTE"? Provide an explanation for your answer.

___
# PART II: UPDATE DATABASE CONTENT

The syntax for the `UPDATE` operation is:

```sql
UPDATE table_name
SET column1 = value1, column2 = value2...., columnN = valueN
WHERE [condition];
```

The syntax for the `INSERT` operation is:

```sql
INSERT INTO table_name [(column1, column2, column3,...columnN)]  
VALUES (value1, value2, value3,...valueN);
```

Database updates are commited automatically in Jupyter/Python. _Transactions_ are an option to control and reverse changes. Additionally we can just reload a backup of the database (NOT an option in deployed systems)

__Note:__ Different to other Database Management Systems, SQLite views are read-only and so you can not execute a `DELETE`, `INSERT` or `UPDATE` statement on a view.

#### Exercise 2.1

Create a table "**RBB**" with the same schema as "**bons_buveurs**" which contains the tuples selected from "**bons_buveurs**"

In [20]:
%%sql DROP TABLE IF EXISTS RBB;

--Create RBB table
CREATE TABLE RBB AS
SELECT *
FROM bons_buveurs
ORDER BY NB;

 * sqlite:///wine.db
Done.
Done.


[]

In [21]:
# Test
%sql SELECT * FROM RBB;

 * sqlite:///wine.db
Done.


[(2, u'Artaud', u'Antonin', u'moyen'),
 (3, u'Aron', u'Raymond', u'gros'),
 (4, u'Apollinaire', u'Guillaume', u'moyen'),
 (6, u'Arrabal', u'Fernando', u'gros'),
 (7, u'Anouilh', u'Jean', u'moyen'),
 (8, u'Aragon', u'Louis', u'gros'),
 (10, u'Andersen', u'Yann', u'gros'),
 (12, u'Bataille', u'Georges', u'moyen'),
 (13, u'Barthes', u'Roland', u'moyen'),
 (14, u'Bory', u'Jean Louis', u'gros'),
 (16, u'Balzac', u'Honore de', u'moyen'),
 (18, u'Celine', u'Louis Ferdinand', u'gros'),
 (19, u'Ciceron', None, u'gros'),
 (20, u'Chateaubriand', u'Francois-Rene de', u'moyen'),
 (22, u'Cocteau', u'Jean', u'gros'),
 (24, u'Courrier', u'Paul Louis', u'moyen'),
 (26, u'Diderot', u'Denis', u'moyen'),
 (27, u'Dumas', u'Alexandre', u'gros'),
 (28, u'Dostoievsky', u'Fiodor', u'gros'),
 (31, u'Dante', None, u'gros'),
 (32, u'Eluard', u'Paul', u'moyen'),
 (33, u'Flaubert', u'Gustave', u'moyen'),
 (35, u'Fromentin', u'Eugene', u'gros'),
 (39, u'Montesquieu', None, u'gros'),
 (42, u'Goethe', u'Johann Wolfgang von', u'moyen'),
 (43, u'Musset', u'Alfred de', u'gros'),
 (46, u'Hugo', u'Victor', u'moyen'),
 (48, u'Hardy', u'Thomas', u'moyen'),
 (50, u'Lautreamont', None, u'moyen'),
 (53, u'Lamartine', u'Alphonse de', u'gros'),
 (55, u'Michelet', u'Jules', u'gros'),
 (57, u'Marx', u'Karl', u'gros'),
 (58, u'Marivaux', u'Pierre Carlet de Chamblain de', u'gros'),
 (60, u'Ovide', None, u'gros'),
 (62, u'Proust', u'Marcel', u'moyen'),
 (64, u'Platon', None, u'gros'),
 (66, u'Perec', u'Georges', u'gros'),
 (68, u'Prevert', u'Jacques', u'gros'),
 (70, u'Paulhan', u'Jean', u'gros'),
 (72, u'Rabelais', u'Francois', u'gros'),
 (73, u'Poe', u'Allan Edgard', u'moyen'),
 (75, u'Ronsard', u'Pierre', u'moyen'),
 (76, u'Racine', u'Jean', u'gros'),
 (78, u'Sandier', u'Jules', u'moyen'),
 (79, u'Sade', u'Donatien Alphonse Fran\xbf\xbfs de', u'gros'),
 (81, u'Sartre', u'Jean-Paul', u'gros'),
 (83, u'Sophocle', None, u'gros'),
 (84, u'Sue', u'Eugene', u'moyen'),
 (86, u'Saint Exupery', u'Antoine de', u'gros'),
 (87, u'Segur', u'Comtesse de', u'gros'),
 (91, u'Verlaine', u'Paul', u'gros'),
 (93, u'Virgile', None, u'gros'),
 (94, u'Zola', u'Emile', u'moyen'),
 (95, u'Zorn', u'Fritz', u'moyen'),
 (98, u'Maturin', u'Robert', u'gros'),
 (99, u'Cesar', u'Jules', u'gros')]

#### Exercise 2.2

Update the table you used to create "**bons_buveurs**": Change the "type" to 'gros' if the total of quantities bought is over 100.

Find the instances to update (schema may differ from the one in your table)

In [22]:
%%sql SELECT NB, NOM, PRENOM, TYPE
FROM Clients2
NATURAL JOIN Sales2
WHERE TYPE != 'gros'
GROUP BY NB
HAVING SUM(QTE) > 100;

 * sqlite:///wine.db
Done.


[(2, u'Artaud', u'Antonin', u'moyen'),
 (5, u'Audiberti', u'Jacques', u'petit'),
 (9, u'Ajar', u'Emile', u'petit'),
 (44, u'Gide', u'Andre', u'petit')]

Update instances

In [23]:
%%sql UPDATE Clients2
SET TYPE = "gros"
WHERE NB IN (SELECT NB
             FROM Sales2
             GROUP BY NB
             HAVING SUM(QTE) > 100)
and TYPE != 'gros';

 * sqlite:///wine.db
4 rows affected.


[]

#### Exercise 2.3

Compare the content of _table_ "**RBB**" and the _view_ "**bons_buveurs**" after the update. What differences do you see? Explain

In [25]:
%%sql SELECT *
FROM RBB
WHERE NB IN (2,5,9,44);

 * sqlite:///wine.db
Done.


[(2, u'Artaud', u'Antonin', u'moyen')]

In [26]:
%%sql SELECT *
FROM bons_buveurs
WHERE NB IN (2,5,9,44);

 * sqlite:///wine.db
Done.


[(2, u'Artaud', u'Antonin', u'gros'),
 (44, u'Gide', u'Andre', u'gros'),
 (9, u'Ajar', u'Emile', u'gros'),
 (5, u'Audiberti', u'Jacques', u'gros')]

#### Exercise 2.4

Create a table "**RBA**" with the same schema as "**buveurs_asec**" which contains the tuples selected from "**buveurs_asec**"

In [37]:
%%sql DROP TABLE IF EXISTS RBA;

--Create RBA table
CREATE TABLE RBA AS
SELECT *
FROM buveurs_asec;

 * sqlite:///wine.db
Done.
Done.


[]

In [41]:
# Test
%sql SELECT * FROM RBA

 * sqlite:///wine.db
Done.


[(11, u'Breton', u'Andre', u'petit'),
 (13, u'Barthes', u'Roland', u'moyen'),
 (16, u'Balzac', u'Honore de', u'moyen'),
 (18, u'Celine', u'Louis Ferdinand', u'gros'),
 (20, u'Chateaubriand', u'Francois-Rene de', u'moyen'),
 (21, u'Corbiere', u'Tristan', u'petit'),
 (23, u'Corneille', u'Pierre', u'petit'),
 (25, u'Char', u'Rene', u'petit'),
 (27, u'Dumas', u'Alexandre', u'gros'),
 (29, u'Fournier', u'Alain', u'petit'),
 (32, u'Eluard', u'Paul', u'moyen'),
 (34, u'Mauriac', u'Francois', u'petit'),
 (35, u'Fromentin', u'Eugene', u'gros'),
 (36, u'Maurois', u'Andre', u'petit'),
 (37, u'Freud', u'Sigmund', u'petit'),
 (39, u'Montesquieu', None, u'gros'),
 (42, u'Goethe', u'Johann Wolfgang von', u'moyen'),
 (43, u'Musset', u'Alfred de', u'gros'),
 (46, u'Hugo', u'Victor', u'moyen'),
 (47, u'Gary', u'Romain', u'petit'),
 (51, u'Lafargue', u'Paul', u'petit'),
 (53, u'Lamartine', u'Alphonse de', u'gros'),
 (54, u'Aurele', u'Marc', u'petit'),
 (55, u'Michelet', u'Jules', u'gros'),
 (56, u'Nerval', u'Gerard de', u'petit'),
 (57, u'Marx', u'Karl', u'gros'),
 (59, u'Nietzsche', u'Friedrich', u'petit'),
 (63, u'Maupassant', u'Guy de', u'petit'),
 (64, u'Platon', None, u'gros'),
 (65, u'Pascal', u'Blaise', u'petit'),
 (66, u'Perec', u'Georges', u'gros'),
 (67, u'Nin', u'Anais', u'petit'),
 (68, u'Prevert', u'Jacques', u'gros'),
 (69, u'Queneau', u'Raymond', u'petit'),
 (70, u'Paulhan', u'Jean', u'gros'),
 (71, u'Radiguet', u'Raymond', u'petit'),
 (72, u'Rabelais', u'Francois', u'gros'),
 (73, u'Poe', u'Allan Edgard', u'moyen'),
 (74, u'Rimbaud', u'Arthur', u'petit'),
 (76, u'Racine', u'Jean', u'gros'),
 (77, u'Rousseau', u'Jean-Jacques', u'petit'),
 (78, u'Sandier', u'Jules', u'moyen'),
 (79, u'Sade', u'Donatien Alphonse Fran\xbf\xbfs de', u'gros'),
 (81, u'Sartre', u'Jean-Paul', u'gros'),
 (83, u'Sophocle', None, u'gros'),
 (84, u'Sue', u'Eugene', u'moyen'),
 (85, u'Segalen', u'Victor', u'petit'),
 (86, u'Saint Exupery', u'Antoine de', u'gros'),
 (87, u'Segur', u'Comtesse de', u'gros'),
 (88, u'Shakespeare', u'William', u'petit'),
 (89, u'Stendhal', None, u'petit'),
 (90, u'Vian', u'Boris', u'petit'),
 (91, u'Verlaine', u'Paul', u'gros'),
 (93, u'Virgile', None, u'gros'),
 (95, u'Zorn', u'Fritz', u'moyen'),
 (98, u'Maturin', u'Robert', u'gros'),
 (99, u'Cesar', u'Jules', u'gros')]

#### Exercise 2.5

Insert a tuple (101, 'your last name', 'your first name', 'your type of purchases(petit, moyen, or gros)') to "**RBA**".

In [42]:
%%sql INSERT INTO RBA (NB, NOM, PRENOM, TYPE)  
VALUES (101, 'Moulin', 'Antoine', 'moyen');

 * sqlite:///wine.db
1 rows affected.


[]

In [43]:
# Test
%sql SELECT * FROM RBA

 * sqlite:///wine.db
Done.


[(11, u'Breton', u'Andre', u'petit'),
 (13, u'Barthes', u'Roland', u'moyen'),
 (16, u'Balzac', u'Honore de', u'moyen'),
 (18, u'Celine', u'Louis Ferdinand', u'gros'),
 (20, u'Chateaubriand', u'Francois-Rene de', u'moyen'),
 (21, u'Corbiere', u'Tristan', u'petit'),
 (23, u'Corneille', u'Pierre', u'petit'),
 (25, u'Char', u'Rene', u'petit'),
 (27, u'Dumas', u'Alexandre', u'gros'),
 (29, u'Fournier', u'Alain', u'petit'),
 (32, u'Eluard', u'Paul', u'moyen'),
 (34, u'Mauriac', u'Francois', u'petit'),
 (35, u'Fromentin', u'Eugene', u'gros'),
 (36, u'Maurois', u'Andre', u'petit'),
 (37, u'Freud', u'Sigmund', u'petit'),
 (39, u'Montesquieu', None, u'gros'),
 (42, u'Goethe', u'Johann Wolfgang von', u'moyen'),
 (43, u'Musset', u'Alfred de', u'gros'),
 (46, u'Hugo', u'Victor', u'moyen'),
 (47, u'Gary', u'Romain', u'petit'),
 (51, u'Lafargue', u'Paul', u'petit'),
 (53, u'Lamartine', u'Alphonse de', u'gros'),
 (54, u'Aurele', u'Marc', u'petit'),
 (55, u'Michelet', u'Jules', u'gros'),
 (56, u'Nerval', u'Gerard de', u'petit'),
 (57, u'Marx', u'Karl', u'gros'),
 (59, u'Nietzsche', u'Friedrich', u'petit'),
 (63, u'Maupassant', u'Guy de', u'petit'),
 (64, u'Platon', None, u'gros'),
 (65, u'Pascal', u'Blaise', u'petit'),
 (66, u'Perec', u'Georges', u'gros'),
 (67, u'Nin', u'Anais', u'petit'),
 (68, u'Prevert', u'Jacques', u'gros'),
 (69, u'Queneau', u'Raymond', u'petit'),
 (70, u'Paulhan', u'Jean', u'gros'),
 (71, u'Radiguet', u'Raymond', u'petit'),
 (72, u'Rabelais', u'Francois', u'gros'),
 (73, u'Poe', u'Allan Edgard', u'moyen'),
 (74, u'Rimbaud', u'Arthur', u'petit'),
 (76, u'Racine', u'Jean', u'gros'),
 (77, u'Rousseau', u'Jean-Jacques', u'petit'),
 (78, u'Sandier', u'Jules', u'moyen'),
 (79, u'Sade', u'Donatien Alphonse Fran\xbf\xbfs de', u'gros'),
 (81, u'Sartre', u'Jean-Paul', u'gros'),
 (83, u'Sophocle', None, u'gros'),
 (84, u'Sue', u'Eugene', u'moyen'),
 (85, u'Segalen', u'Victor', u'petit'),
 (86, u'Saint Exupery', u'Antoine de', u'gros'),
 (87, u'Segur', u'Comtesse de', u'gros'),
 (88, u'Shakespeare', u'William', u'petit'),
 (89, u'Stendhal', None, u'petit'),
 (90, u'Vian', u'Boris', u'petit'),
 (91, u'Verlaine', u'Paul', u'gros'),
 (93, u'Virgile', None, u'gros'),
 (95, u'Zorn', u'Fritz', u'moyen'),
 (98, u'Maturin', u'Robert', u'gros'),
 (99, u'Cesar', u'Jules', u'gros'),
 (101, u'Moulin', u'Antoine', u'moyen')]

#### Exercise 2.6

Compare the content of _table_ "**RBA**" and the _view_ "**buveurs_asec**". What differences do you see? Explain

In [45]:
%%sql SELECT *
FROM RBA
WHERE NB = 101

 * sqlite:///wine.db
Done.


[(101, u'Moulin', u'Antoine', u'moyen')]

In [46]:
%%sql SELECT *
FROM buveurs_asec
WHERE NB = 101

 * sqlite:///wine.db
Done.


[]

___
# PART III: Design the database schema for posts in a Social Network

In this section your task is to design the database schema for a social network app of a new startup:

The new social network will contain users, where each user will have a name, a nickname, an email, date of birth, and an address (Street, City, State, Country, Postal Code). Users can be friends of other users, and can publish posts. Each post can contain a text, date and attachment. Posts can be either original posts or replies so the app needs to handle both scenarios. When users log in, the app needs to display the posts of their friends.

**Note:** You can create diagrams of your proposal and insert them as images into this notebook.

#### Exercise 3.1

Write and explain the design of the relations of your database

Answer: We define the following tables :

<center> **Users** </center>

|  *Attribute*  |         *Description*          |
|---------------|--------------------------------|
| ID            | ID of the user                 |
| NAME          | Name of the user               |
| NICKNAME      | Nickname of the user           |
| EMAIL         | Email of the user              |
| DATE_OF_BIRTH | Date of Birth of the user      |
| STREET        | Street where the user lives    |
| POSTAL_CODE   | Postal code of the user's city |

The attribute ID is a key for the table Users. 

Assuming the street with the postal code entirely define the user's adress, we only keep the attributes STREET and POSTAL_CODE in our table Users, and we define a new table Location, with the attributes STREET, CITY, STATE, COUNTRY & POSTAL_CODE. Thus, the attributes (STREET, POSTAL_CODE) of the table Users are a foreign key for the table Location defined below.


<center> **Location** </center>

|  *Attribute*  |         *Description*          |
|---------------|--------------------------------|
| STREET        | Street where the user lives    |
| CITY          | City where the user lives      |
| STATE         | State where the user lives     |
| COUNTRY       | Country where the user lives   |
| POSTAL_CODE   | Postal code of the user's city |

Besides, we define the table Friendship to store all relations between users. If (ID_1, ID2) is in the table Friendship, it means that the user with ID_1 is friend with the user with ID_2. Obviously, the table also contains (ID_2, ID_1). Both attributes ID1 or ID2 are a foreign key for the table Users.


<center> **Friendship** </center>

|  *Attribute*  |         *Description*          |
|---------------|--------------------------------|
| ID1           | ID of the user                 |
| ID2           | ID of the user's friend        |

Finally, we define the table Posts with the information required.

<center> **Posts** </center>

|*Attribute*|         *Description*          |
|-----------|--------------------------------|
| ID        | ID of the post                 |
| TEXT      | Content of the post            |
| DATE      | Date of publication            |
| ATTACHMENT| Attachment                     |
| ORIGINAL  | Is it a original post ?        |


#### Exercise 3.2

Write a view to retrieve the posts to display when a user logs in. Consider that some users may have a lot of friends and you need to limit the number of post to display. How would you select relevant posts to display first? What kind of information would you use/add in the database for this purpose? Explain your answer.

__Note:__ Limiting the number of posts just by count is too simplistic, the user could be missing something interesting to him/her.

Answer: Considering only the data we have, we could slightly modify the attribute ORIGINAL of the table Posts and say that this attribute is equal to the number of times the posts has been taken by an user, e.g. 4 if it has 4 replies, 0 if it is original. Then, we could sort the posts thanks to the ORIGINAL attribute.

Another idea could be to add a NB_VIEWS attribute to the Posts table, that counts the number of time the post has been seen. We could also add a NB_LIKES just like Facebook ...

Anyway, given an specific ID id0, such a view could look like :

%%sql
DROP VIEW IF EXISTS sort_posts;

CREATE VIEW sort_posts AS
SELECT *
FROM Posts
JOIN Friendship ON Posts.ID = Friendship.ID1
WHERE Friendship.ID2 = id0
ORDER BY ORIGINAL
LIMIT 10;